# Create a tasking order via UP42 SDK



## Installation

[Install](https://sdk.up42.com/installation/) the SDK. The package requires Python ≥ 3.7.

In [ ]:
pip install up42-py
import up42

## Authentication

To authenticate, you need to know your project credentials. Read more about authentication in the [documentation](https://sdk.up42.com/authentication).

Paste your credentials into the cell below.

In [4]:
up42.authenticate(
    project_id="your-project-ID",
    project_api_key="your-project-API-key"
)

# Alternatively, you can create a config.json file and use the following code for it:
# {
#     "project_id": your-project-ID",
#     "project_api_key": "your-project-API-key"
# }

# If you use a configuration file, uncomment this line:
# up42.authenticate(cfg_file="credentials.json")


2023-07-13 14:28:50,176 - Authentication with UP42 successful!


Place an order as follows:

1. Choose a tasking collection
2. Request access
3. Accept a EULA
4. Get a JSON schema of an order form
5. Fill out an order form
6. Place an order
7. Review and accept feasibility
8. Activate an order
9. Monitor an order

In [6]:
# Let's get a list of all the satellites you can currently task
tasking = up42.initialize_tasking()
tasking.get_data_products(basic=True)

{'SPOT tasking': {'collection': 'SPOT-tasking',
  'host': 'oneatlas',
  'data_products': {'Custom': '3a89d24e-515a-460f-a494-96be55da10a9',
   'Analytic': '9d16c506-a6c0-4cf2-a020-8ecaf10b4160',
   'Display': '47a693ba-4f8b-414d-8d5b-697373df4765'}},
 'Pléiades Neo tasking': {'collection': 'pneo-tasking',
  'host': 'oneatlas',
  'data_products': {'Custom': '07c33a51-94b9-4509-84df-e9c13ea92b84',
   'Analytic (Mono)': '123eabab-0511-4f36-883a-80928716c3db',
   'Display (Mono)': '469f9b2f-1631-4c09-b66d-575abd41dc8f'}},
 'Capella Space tasking': {'collection': 'capella-tasking',
  'host': 'capellaspace',
  'data_products': {'Custom': '83e21b35-e431-43a0-a1c4-22a6ad313911'}},
 'Pléiades tasking': {'collection': 'PHR-tasking',
  'host': 'oneatlas',
  'data_products': {'Custom': '4f866cd3-d816-4c98-ace3-e6105623cf13',
   'Analytic': 'bd102407-1814-4f92-8b5a-7697b7a73f5a',
   'Display': '28d4a077-6620-4ab5-9a03-c96bf622457e'}},
 'TerraSAR-X tasking': {'collection': 'terra-sar-tasking',
  'ho

In [7]:
# Let's task a PHR satellite
# First let's define geometry. There are multiple ways to define a geometry
# You can simply use one of your existing vector file
target_geometry = up42.read_vector_file("aoi_washington.geojson")

# OR, by defining the geometry in the code iteself
target_geometry = {
    "type": "Polygon",
    "coordinates": (
        (
            ( 13.362774607378753,
              52.52381077911102
            ),
            (
              13.362774607378753,
              52.501421984603894
            ),
            (
              13.414800488880843,
              52.501421984603894
            ),
            (
              13.414800488880843,
              52.52381077911102
            ),
            (
              13.362774607378753,
              52.52381077911102
            )
        ),
    ),
}

In [ ]:
# Let's define additional order parameters

order_parameters = tasking.construct_order_parameters(
    data_product_id="bd102407-1814-4f92-8b5a-7697b7a73f5a",
    name="A demo example to task a PHR satellite",
    acquisition_start="2023-08-01",
    acquisition_end="2023-10-07",
    geometry=target_geometry,
    tags=["example", "now-you-can-add-tags-to-the-order"]
)

In [ ]:
# This will return you a list of some additional parameters that you need to configure
# This list changes from satellite to satellite
# 2023-07-13 11:23:58,962 - As `acquisitionMode` select one of ['mono', 'stereo', 'tristereo']
# 2023-07-13 11:23:58,962 - As `cloudCoverage` select `{'description': 'Maximum allowed cloud coverage in percentage.', 'minimum': 5, 'maximum': 20, 'type': 'integer'}`
# 2023-07-13 11:23:58,962 - As `incidenceAngle` select `{'description': 'Maximum allowed incidence angle in degree.', 'minimum': 5, 'maximum': 50, 'type': 'integer'}`
# 2023-07-13 11:23:58,962 - As `geometricProcessing` select one of ['ortho']
# 2023-07-13 11:23:58,962 - As `spectralProcessing` select one of ['bundle']
# 2023-07-13 11:23:58,962 - As `pixelCoding` select one of ['12bits']
# 2023-07-13 11:23:58,962 - As `radiometricProcessing` select one of ['reflectance']
# 2023-07-13 11:23:58,962 - As `deliveredAs` select one of ['geotiff']

In [ ]:
# Update the request by adding the requested parameters, for example:
order_parameters["params"].update({
    "acquisitionMode": "mono",
    "cloudCoverage": 10,
    "incidenceAngle": 10,
    "geometricProcessing": "ortho",
    "spectralProcessing": "bundle",
    "pixelCoding": "12bits",
    "radiometricProcessing": "reflectance",
    "deliveredAs": "geotiff",
    "projection": "4326"
})

In [ ]:
# Finally, let's place the order
order = tasking.place_order(order_parameters)

In [ ]:
# One the order is placed, you will get a response like
# 2023-07-13 13:22:10,677 - Order is PLACED
# 2023-07-13 13:22:10,677 - Order your-order-id is now PLACED.

In [ ]:
# After an order is placed, the Operations team will look at the order feasibility. They will evaluate the tasking parameters with the provider, and then will present the following assessments:

# The order is possible with the given parameters.
# The order requires modifications with suitable options proposed.
# You will receive an email notifying you when your order feasibility has been assessed.

# Using your order ID, review the proposed feasibility options as follows:
feasibilities = tasking.get_feasibility(order_id="your-order-id")


In [ ]:
# This will return a list of all possible feasibility studies. For example:
"""
{
            "id": "some-unique-id",
            "createdAt": "2023-05-09T10:20:10.520992Z",
            "updatedAt": "2023-05-09T10:20:10.520992Z",
            "accountId": "account-id",
            "workspaceId": "workspace-id",
            "orderId": "order-id",
            "type": "MANUAL",
            "options": [
                {
                    "id": "some-unnique-id",
                    "createdAt": "2023-05-09T10:20:11.203893Z",
                    "updatedAt": "2023-05-09T10:20:11.203893Z",
                    "studyId": "c1cc719f-e429-4bfa-adc3-c4922489adfe",
                    "description": "test feasibility"
                }
            ],
            "decision": "NOT_DECIDED",
            "decisionById": null,
            "decisionByType": null,
            "decisionAt": null,
            "decisionOption": null
}
"""

In [ ]:
# Let's select one of the options:
tasking.choose_feasibility(
    feasibility_id="feasibility-id",
    accepted_option_id="accepted-option-id"
)

In [ ]:
# After selecting a feasibility option, you will receive a price quote for your order. Using your order ID, review the received quotation as follows:
tasking.decide_quotation(
    quotation_id="quotation-id",
    decision="ACCEPTED"
)

In [ ]:
# You can check the status of your order by
order.status